In [7]:
import os

from dotenv import load_dotenv
import praw
load_dotenv()

# print all environment variables
secrets = ["REDDIT_CLIENT_ID", "REDDIT_CLIENT_SECRET", "REDDIT_USER_AGENT", "REDDIT_USERNAME", "REDDIT_PASSWORD"]
for secret in secrets:
    print(f"{secret}: {os.getenv(secret)}")

REDDIT_CLIENT_ID: iPrv4qb5mBu_JqR__dSU3w
REDDIT_CLIENT_SECRET: kHJlE6zMQADWXLNi1tShZQ48eVblMw
REDDIT_USER_AGENT: red-it-meter (by u/Fabulous_Cherry2510)
REDDIT_USERNAME: Fabulous_Cherry2510
REDDIT_PASSWORD: MASSNEch_03


In [10]:
try:
    reddit = praw.Reddit(
        client_id = os.getenv("REDDIT_CLIENT_ID"),
        client_secret = os.getenv("REDDIT_CLIENT_SECRET"),
        password = os.getenv("REDDIT_PASSWORD"),
        user_agent = os.getenv("REDDIT_USER_AGENT"),
        username = os.getenv("REDDIT_USERNAME")
    )
except Exception as e:
    print(f"Error initializing Reddit client: {e}")

print(reddit.user.me())

Fabulous_Cherry2510


In [22]:
subreddit = reddit.subreddit("all")
posts = subreddit.hot(limit=10)

In [23]:
posts = list(posts)

In [24]:
for post in posts:
    print(f"Title: {post.title}")
    print(f"Score: {post.score}")
    print(f"URL: {post.url}")
    print(f"Author: {post.author}")
    print(f"Created at: {post.created_utc}")
    print("-" * 40)

Title: Self-Inflicted Harm...
Score: 31646
URL: https://i.redd.it/ed24viwnep8f1.jpeg
Author: CapitanJackSparow-33
Created at: 1750696210.0
----------------------------------------
Title: Old age and treachery
Score: 43474
URL: https://v.redd.it/q836wk53dp8f1
Author: Boundaries1st
Created at: 1750695769.0
----------------------------------------
Title: Tooter is looking awfully polite
Score: 20289
URL: https://www.reddit.com/gallery/1lilimr
Author: ShevaunA
Created at: 1750696905.0
----------------------------------------
Title: Maybe he was
Score: 25917
URL: https://i.redd.it/b0aids29ep8f1.jpeg
Author: IdeaOrdinary48
Created at: 1750696074.0
----------------------------------------
Title: What is it? The braids?
Score: 15458
URL: https://i.redd.it/8zbtq97ulp8f1.jpeg
Author: ThickCapital
Created at: 1750698621.0
----------------------------------------
Title: Aruba sun got me
Score: 11661
URL: https://i.redd.it/ufucrz2mtp8f1.jpeg
Author: dogmom4life13
Created at: 1750701236.0
----------

In [25]:
popular = reddit.subreddits.popular(limit=100)
for sub in popular:
    print(sub.display_name)

Home
AskReddit
NoStupidQuestions
BaldursGate3
facepalm
interestingasfuck
Damnthatsinteresting
LivestreamFail
pics
Palworld
AmItheAsshole
mildlyinfuriating
Piracy
PeterExplainsTheJoke
funny
AITAH
movies
Helldivers
gaming
worldnews
leagueoflegends
pcmasterrace
Unexpected
news
politics
wallstreetbets
todayilearned
nottheonion
memes
PublicFreakout
Wellthatsucks
explainlikeimfive
OutOfTheLoop
OnePiece
BlackPeopleTwitter
buildapc
HonkaiStarRail
SipsTea
Minecraft
mildlyinteresting
nfl
BeAmazed
DIY
nba
MapPorn
Steam
Overwatch
Genshin_Impact
classicwow
soccer
Eldenring
badroommates
personalfinance
antiwork
anime
wow
DnD
technology
WhitePeopleTwitter
EscapefromTarkov
unpopularopinion
popculturechat
videos
BestofRedditorUpdates
youtube
legaladvice
ffxiv
sysadmin
MadeMeSmile
CombatFootage
relationship_advice
discordapp
pcgaming
Games
ChatGPT
GlobalOffensive
2007scape
formula1
CuratedTumblr
Gamingcirclejerk
TikTokCringe
PiratedGames
techsupport
shitposting
theydidthemath
malelivingspace
WTF
cyberpu